In [1]:
pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118

Looking in indexes: https://download.pytorch.org/whl/cu118
Note: you may need to restart the kernel to use updated packages.


In [2]:
#pip install pandas

In [3]:
pip install transformers

Note: you may need to restart the kernel to use updated packages.


In [4]:
import pandas as pd

# Load the CSVs
health_df = pd.read_csv(r"data\tarot_readings_health_final.csv")
love_df = pd.read_csv(r"data\tarot_readings_love_final.csv")
profession_df = pd.read_csv(r"data\tarot_readings_profession_final.csv")

# Add a column to identify the category
health_df["category"] = "health"
love_df["category"] = "love"
profession_df["category"] = "profession"

In [5]:
df = pd.concat([health_df, love_df, profession_df], ignore_index=True)

In [6]:
from transformers import pipeline

c:\Users\Jue\Desktop\NLP_Final_Project\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [7]:
df.columns

Index(['zodiac', 'question', 'card1', 'card2', 'card3', 'reading', 'category'], dtype='object')

In [8]:
# Load the sentiment analysis pipeline
sentiment_analyzer = pipeline("sentiment-analysis")

# Analyze sentiments
df["question_sentiment"] = df["question"].apply(lambda x: sentiment_analyzer(x)[0]["label"])
df["reading_sentiment"] = df["reading"].apply(lambda x: sentiment_analyzer(x)[0]["label"])

No model was supplied, defaulted to distilbert/distilbert-base-uncased-finetuned-sst-2-english and revision 714eb0f (https://huggingface.co/distilbert/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.
Device set to use cuda:0
You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


In [2]:
import pandas as pd
df = pd.read_csv("tarot_readings_with_sentiment.csv")

In [4]:
df

,zodiac,question,card1,card2,card3,reading,category,question_sentiment,reading_sentiment
0,Cancer,How can I cultivate more self-care in my life?,The Emperor,Nine of Pentacles,Strength,"Reading: Cancer, The Emperor suggests establi...",health,NEGATIVE,POSITIVE
1,Pisces,How can I release fear about my health?,The Chariot,Death,Four of Cups,"Reading: Pisces, The Chariot suggests taking ...",health,NEGATIVE,NEGATIVE
2,Aquarius,Is it time to seek medical help or a second op...,The Hanged Man,Seven of Swords,Wheel of Fortune,"Aquarius, the cards suggest a pause is needed ...",health,NEGATIVE,NEGATIVE
3,Virgo,Will I recover fully from this illness?,Ten of Cups,The Chariot,The Sun,"Virgo, your cards point to a positive outcome ...",health,NEGATIVE,POSITIVE
4,Leo,How can I better support my physical well-being?,Two of Swords,Nine of Wands,The Tower,"Reading: Leo, the Two of Swords suggests you'...",health,NEGATIVE,NEGATIVE
...,...,...,...,...,...,...,...,...,...
5988,Pisces,What does the future hold for my career?,Two of Wands,Six of Pentacles,Knight of Pentacles,"Reading: Pisces, you're at a crossroads with ...",profession,NEGATIVE,POSITIVE
5989,Capricorn,How can I be more confident in my career path?,Nine of Wands,Five of Cups,Five of Wands,"Reading: Capricorn, you're almost there, but ...",profession,NEGATIVE,POSITIVE
5990,Aries,How can I shine more in my workplace?,Knight of Wands,The Tower,The Chariot,Reading: The Knight of Wands encourages you t...,profession,POSITIVE,POSITIVE
5991,Capricorn,Should I invest in further education or training?,Page of Wands,Nine of Wands,Six of Cups,Reading: The Page of Wands sparks a new inter...,profession,NEGATIVE,POSITIVE


In [5]:
df.to_csv("tarot_readings_with_sentiment.csv", index=False)

In [6]:
#pip install datasets

In [7]:
df.columns

Index(['zodiac', 'question', 'card1', 'card2', 'card3', 'reading', 'category',
       'question_sentiment', 'reading_sentiment'],
      dtype='object')

In [8]:
from datasets import Dataset


# Create Flan-T5 style input and target columns (without category)
df["input"] = (
    "Zodiac: " + df["zodiac"] +
    " | Sentiment: " + df["question_sentiment"] +
    " | Cards: " + df["card1"] + ", " + df["card2"] + ", " + df["card3"] +
    " | Question: " + df["question"]
)

# Target column is the reading
df["target"] = df["reading"]

# Drop missing rows (if any)
df = df[["input", "target"]].dropna()

# Convert to Hugging Face Dataset
hf_dataset = Dataset.from_pandas(df)

c:\Users\Jue\Desktop\NLP_Final_Project\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [9]:
#pip install sentencepiece

In [11]:
from transformers import T5Tokenizer

# Load tokenizer
tokenizer = T5Tokenizer.from_pretrained("google/flan-t5-base")

# Tokenization function
def preprocess(example):
    inputs = tokenizer(
        example["input"],
        max_length=512,
        padding="max_length",
        truncation=True
    )
    targets = tokenizer(
        example["target"],
        max_length=128,
        padding="max_length",
        truncation=True
    )
    inputs["labels"] = targets["input_ids"]
    return inputs

# Apply tokenization
tokenized_dataset = hf_dataset.map(preprocess, batched=True)



Map: 100%|██████████| 5993/5993 [00:02<00:00, 2923.12 examples/s]


In [12]:
pip install --upgrade transformers

Note: you may need to restart the kernel to use updated packages.


In [13]:
pip install "accelerate>=0.26.0"

Note: you may need to restart the kernel to use updated packages.


In [14]:
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir="./flan-t5-tarot",
    per_device_train_batch_size=8,
    num_train_epochs=3,
    learning_rate=3e-4,
    weight_decay=0.01,
    logging_dir="./logs",
    logging_steps=50,
    save_steps=500
    # Don't use "report_to", "evaluation_strategy", or "save_total_limit" in older versions
)


In [15]:
from transformers import Trainer, T5ForConditionalGeneration

# Load model again if needed
model = T5ForConditionalGeneration.from_pretrained("google/flan-t5-base")

# Define the Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
)

# Start training
trainer.train()


Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.48.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.


RuntimeError: CUDA error: CUBLAS_STATUS_EXECUTION_FAILED when calling `cublasSgemm( handle, opa, opb, m, n, k, &alpha, a, lda, b, ldb, &beta, c, ldc)`